In [ ]:
using Pkg; Pkg.activate(".")
using Plots, LinearAlgebra, LaTeXStrings, ForwardDiff
include("tools.jl")

# Pseudo-Spectral Methods

(collocation methods)

## Example 1: transport with variable coefficients

$$
\begin{aligned}
  u_t + v(x) u_x = 0, \qquad \text{(+PBC)}
\end{aligned}
$$
with initial condition 
$$
    u(0, x) = \exp( - 10 * (1-\cos(x))/2)
$$

We discretize this in time with the Leapfrog scheme and in space with a pseudo-spectral method (details see class) which results in 
$$
	\frac{u_N^{n+1} - u_N^{n-1}}{2 \Delta t} + I_N[ v \cdot u_{N,x}^n ]  = 0.
$$

In [ ]:
function plot_soln(t, X, u)
    plot( xaxis = ([0, 2*π], ), yaxis = ([-0.1, 1.2],) , size = (400, 250), 
          legend = :topright, 
          title = "t = $(round(t, digits=2))")
    plot!(X, V, lw=3, label = "v(x)")
    plot!(X, U, lw=3, label = "u(t, x)")
end

# ------------------------------------------
# Problem setup
N = 128  
dt = π/(6N) 
tmax = 32.0
vfun = x -> 0.33 + sin(x/2)^2 / 2
u0 = x ->  exp(- 10 * (1+cos(x))/2 )
#------------------------------------------


X = xgrid(N)
K = kgrid(N)
V = vfun.(X)

# transport operator: x -> k -> x 
trans = let K=K, V=V
    U -> V .* real.(ifft(im * (K .* fft(U))))
end

# initial condition, we also need one additional v in the past
# (this takes one step of the PDE backward in time)
U = u0.(X)
Uold = U - dt * trans(U)

# time-stepping loop
@gif for t = 0:dt:tmax
    global U, Uold, trans 
    U, Uold = Uold - 2 * dt * trans(U), U
    plot_soln(t, X, U)
end every 20


---

### Example 2: Elliptic PDE

$$
 Lu = - {\rm div} A(x) \nabla u(x) + c(x) u = f, \qquad \text{(+ PBC)}
$$


In [ ]:
using StaticArrays

# model specification
A_fun = (x1, x2) -> SA[1+0.5*sin(x1) 0.25*cos(x1+x2); 0.25*cos(x1+x2) 1+0.5*sin(x2) ]
c_fun = (x1, x2) -> 1+sin(cos(x1)*cos(x2))
f_fun = (x1, x2) -> exp(-10*(2+cos(x1)+cos(x2)))

# discretization parameters 
D = 2
N = 32

# differential equation L[U] = F
L, F = let A_fun = A_fun, c_fun = c_fun, f_fun = f_fun, D = D, N = N
    X1, X2 = xgrid(D, N)
    K1, K2 = kgrid(D, N) 
    F = f_fun.(X1, X2)
    C = c_fun.(X1, X2) 
    A = A_fun.(X1, X2)
    A11 = getindex.(A, 1, 1)
    A12 = getindex.(A, 1, 2)
    A22 = getindex.(A, 2, 2)

    L = U -> begin
        Û = fft(U) 
        # in k-space we can evaluate ∇u_N
        ∇U1 = real.(ifft(im * K1 .* Û))
        ∇U2 = real.(ifft(im * K2 .* Û))
        # multiply with diffusion coeff: q = I_N[ A * ∇u_N ]
        Q1 = A11 .* ∇U1 + A12 .* ∇U2
        Q2 = A12 .* ∇U1 + A22 .* ∇U2 
        # apply div
        divQ = real.(ifft( im * K1 .* fft(Q1) + im * K2 .* fft(Q2) ))
        return - divQ + C .* U
    end
    
    (L, F)
end
;

In [ ]:
# We need to solve L(U) = F 
# OPTION 1: 

# (1) Assemble the dense linear system!
L_mat = zeros((2*N)^2, (2*N)^2)
for i = 1:(2*N)^2
    ei = zeros(2*N, 2*N)
    ei[i] = 1
    L_mat[:, i] = L(ei)[:]
end
L_mat

In [ ]:
# (2) Solve with dense linear algebra 
U = reshape( L_mat \ F[:], (2*N, 2*N) )

# just to confirm that we really did solve the right system
@show norm( (L(U) - F)[:], Inf )

In [ ]:
# not a very interesting solution, but might as well plot it
xp = xgrid(N) 
surface(xp, xp, U, cb=nothing, size = (300, 300))

In [ ]:

# OPTION 2: iterative solver 

using IterativeSolvers, LinearMaps

L_vec = u -> L(reshape(u, 2*N, 2*N))[:]
L_map = LinearMap(L_vec, L_vec, (2*N)^2)
u = IterativeSolvers.cg(L_map, F[:], abstol=1e-3, reltol=1e-3)
U_cg = reshape(u, 2*N, 2*N)

@show norm(u - U[:], Inf)
@show norm( (L(U)    - F)[:], Inf )
@show norm( (L(U_cg) - F)[:], Inf );

In [ ]:
# One can save a lot of time here; even more in 3D!!!
# and much much more if one pays more attention to performance. 
@time IterativeSolvers.cg(L_map, F[:], abstol=1e-3, reltol=1e-3);
@time L_mat \ F[:];